In [1]:
import os, sys

import torch
import numpy as np


In [2]:
os.chdir('/home/hugal/AIC/WIA/Ourseq2seq/')
import seq2seq

#Loading the dataset

In [3]:

test_src = open('/home/hugal/AIC/WIA/writingPrompts/test.wp_source', 'r')
test_tgt = open('/home/hugal/AIC/WIA/writingPrompts/test.wp_target', 'r')

tst_src = test_src.readlines()
tst_tgt = test_tgt.readlines()

test_src.close()
test_tgt.close()

In [4]:
print(len(tst_src))
print(len(tst_tgt))

15138
15138


In [5]:
for i in range(len(tst_src)):
    tst_src[i] = tst_src[i].split()[3:]
    tst_tgt[i] = tst_tgt[i].split()[3:]

In [6]:
def voc(data):
    dict = {}
    for l in data:
        for w in l:
            if w not in dict.keys():
                dict[w] = len(dict)
    return dict

def convert(data, vocab):
    d = []
    for l in data:
        c = []
        for w in l:
            c.append(vocab[w])
        c.reverse()
        d.append(c)
    return d

def padding(data_c, pad, ma):
    l = [len(e) for e in data_c]
    m = min(np.amax(l), ma)
    print(m)
    for i in range(len(l)):
        if l[i] > m:
            l[i] = m
            data_c[i] = data_c[i][:m]
        else:
            for j in range(m-l[i]):
                data_c[i].append(pad)
    return l, data_c
    

In [7]:
vocab = voc(tst_src + tst_tgt)

tst_src_c = convert(tst_src, vocab)
tst_tgt_c = convert(tst_tgt, vocab)

pad = '<pad>'
sos = '<sos>'
eos = '<eos>'
vocab[pad] = len(vocab)
vocab[sos] = len(vocab)
vocab[eos] = len(vocab)
tst_tgt_c[:].insert(0, vocab[sos])
tst_tgt_c[:].append(vocab[eos])
l_src, tst_src_p = padding(tst_src_c, vocab[pad], 1000)
l_tgt, tst_tgt_p = padding(tst_tgt_c, vocab[pad], 1000)

z = list(zip(l_src, tst_src_p, l_tgt, tst_tgt_p))
z.sort(reverse=True)
l_src, tst_src_p, l_tgt, tst_tgt_p = zip(*z)

tst_src_t = torch.LongTensor(tst_src_p)
tst_tgt_t = torch.LongTensor(tst_tgt_p)

72
1000


In [8]:
print(len(vocab))

143097


In [9]:
print(tst_src_p[:5])
print(l_src[:5])

([22, 629, 333, 5, 287, 474, 83, 82, 688, 962, 85, 191, 390, 403, 312, 399, 1808, 22, 82, 141, 8, 838, 797, 895, 424, 59, 4029, 421, 2, 609, 6999, 3, 73, 170, 107, 311, 797, 403, 73, 240, 226, 403, 82, 312, 539, 103, 5282, 17, 8352, 358, 59, 4389, 3, 191, 2933, 333, 141, 17, 341, 93, 5, 2703, 37, 191, 459, 1206, 118, 107, 227, 226, 2670, 103], [111, 1163, 421, 3694, 494, 3087, 474, 59, 3038, 20, 3, 502, 425, 1283, 343, 22, 1441, 93, 550, 494, 3087, 474, 59, 3038, 865, 3, 502, 367, 155, 22, 4121, 48, 3552, 160, 1453, 5, 121, 357, 59, 72, 686, 2, 479, 22, 1630, 3, 341, 8, 59, 8296, 59, 672, 59, 4417, 59, 2655, 59, 1172, 59, 4243, 22, 72, 37, 2, 10025, 5, 3428, 1220, 320, 1195, 143094, 143094], [226, 1477, 100, 710, 411, 22, 4456, 5914, 5538, 2, 1865, 37, 19, 2199, 3, 5913, 396, 59, 967, 396, 123, 22, 3364, 5912, 2934, 1965, 3, 61, 5050, 10, 28, 2203, 23, 22, 2603, 37, 216, 609, 5911, 2132, 5910, 23, 22, 5909, 5908, 5907, 3, 17, 520, 5906, 37, 19, 4395, 650, 515, 5905, 3, 2888, 23, 22, 26

# Training

In [10]:
enc = seq2seq.Encoder(len(vocab), 20, 50, 1, 'cuda', vocab[pad])
dec = seq2seq.Decoder(len(vocab), 20, 50, 1, 'cuda', vocab[pad],vocab[sos], vocab[eos])

In [11]:
enc.to('cuda')
dec.to('cuda')
opt_enc = torch.optim.SparseAdam(enc.parameters())
opt_dec = torch.optim.SparseAdam(dec.parameters())

RuntimeError: cuda runtime error (30) : unknown error at /opt/conda/conda-bld/pytorch_1533672544752/work/aten/src/THC/THCGeneral.cpp:74

In [ ]:
#out, h_n = enc(tst_src_t[:4], torch.LongTensor(l_src[:4]))
#output, h = dec(h_n, 50, batch_size=4)

In [ ]:
#out

In [ ]:
'''
s = []
v_k = list(vocab.keys())
v_v = list(vocab.values())
for e in output[0]:
    s.append(v_k[v_v.index(e)])
print(s)

In [ ]:
opt_dec.zero_grad()
opt_enc.zero_grad()

In [ ]:
lengths = torch.LongTensor(l_src[:4])
out, h_n = enc(tst_src_t[:4], lengths)
output = dec.teacher_force(tst_tgt_t[:4], h_n)

In [ ]:
loss = 0
for o, l, t in zip(output.transpose(0,1), l_tgt[:4], tst_tgt_t[:4]):
    loss += criterion(o[:l], t[1:(l+1)])/(l*4)

#Saving weights

#Generating

#Saving generated examples